In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions

from bs4 import BeautifulSoup
import re

import sys
if 'ipykernel' in sys.modules:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

import pandas as pd
import time
from tqdm import tqdm

In [5]:
#영화 리스트 가져오기

df = pd.read_csv('17-23movis.csv')
df

,영화제목,개봉일,매출액,매출액.1,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,주연,영화소개
0,스파이더맨: 노 웨이 홈,2021-12-15,"53,772,689,910",4.9%,"53,772,689,910","5,369,773",5369773,"2,948",137848,미국,미국,NaN,소니픽쳐스엔터테인먼트코리아주식회사극장배급지점,12세이상관람가,"액션,어드벤처,SF",존 왓츠,"톰 홀랜드,젠데이아 콜먼,베네딕트 컴버배치,제이콥 배덜런,존 파브로,마리사 토메이",NaN
1,남산의 부장들,2020-01-22,"41,225,216,650",3.8%,"41,225,216,650","4,750,345",4750345,"1,659",140051,한국,한국,(주)하이브미디어코프,(주)쇼박스,15세이상관람가,드라마,우민호,"이병헌,이성민,곽도원,이희준,김소진,김승훈,박성근,서현우,지현준,박지일,이태형,주석...",NaN
2,다만 악에서 구하소서,2020-08-05,"38,602,260,990",3.5%,"38,602,260,990","4,357,803",4357803,"1,998",193842,한국,한국,(주)하이브미디어코프,(주)씨제이이엔엠,15세이상관람가,"범죄,액션",홍원찬,"황정민,이정재,박정민,박소이,최희서,송영창,이서환,박명훈,오대환,심영은,김성곤,최정...",NaN
3,반도,2020-07-15,"33,073,948,880",3.0%,"33,073,948,880","3,812,455",3812455,"2,575",199084,한국,한국,(주)영화사레드피터,(주)넥스트엔터테인먼트월드(NEW),15세이상관람가,"액션,드라마",연상호,"강동원,이정현,권해효,김민재,구교환,김도윤,이레,이예원,김규백,황영희,장소연,문우진...",NaN
4,모가디슈,2021-07-28,"34,558,297,730",3.1%,"34,558,297,730","3,613,984",3613984,"1,688",210740,한국,한국,"(주)덱스터스튜디오,(주)외유내강,(주)필름케이",롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,"액션,드라마",류승완,"김윤석,조인성,허준호,구교환,김소진,정만식,김재화,박경혜,박명신,한철우,주보비,안세...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3560,항거:유관순 이야기,2019-02-27,"2,522,900",0.0%,"9,293,982,001",295,1158248,44,105,한국,한국,(주)디씨지플러스,롯데컬처웍스(주)롯데엔터테인먼트,12세이상관람가,드라마,조민호,"고아성,김새벽,김예은,정하담,류경수,최무성,하성광,류성민,권현진,김경록,김남진,박지...",NaN
3561,데쓰 캘린더,2022-06-23,"2,492,500",0.0%,"2,492,500",264,264,17,115,프랑스,프랑스,NaN,THE 픽쳐스,15세이상관람가,"공포(호러),스릴러",NaN,NaN,NaN
3562,포프란: 사라진 X를 찾아서,2022-10-27,"2,438,400",0.0%,"2,438,400",259,259,17,165,일본,일본,NaN,(주)이놀미디어,15세이상관람가,코미디,우에다 신이치로,NaN,NaN
3563,대명금의위,2022-04-06,"258,000",0.0%,"258,000",138,138,5,108,중국,중국,NaN,(유)조이앤시네마,15세이상관람가,"액션,스릴러",NaN,NaN,NaN


In [12]:
df

,영화제목,Unnamed: 1
0,천문: 하늘에 묻는다,NaN
1,테넷,NaN
2,시실리 2km,NaN
3,크루엘라,NaN
4,육사오(6/45),NaN
...,...,...
428,노아,NaN
429,레버넌트: 죽음에서 돌아온 자,NaN
430,오페라의 유령,NaN
431,테이큰 3,NaN


In [6]:
# pd파일 읽어서 영화명 리스트로 만들기
df = pd.read_csv('17-23movis.csv')
movie_list = df['영화제목'].tolist()
movie_list  

['스파이더맨: 노 웨이 홈',
 '남산의 부장들',
 '다만 악에서 구하소서',
 '반도',
 '모가디슈',
 '이터널스',
 '블랙 위도우',
 '히트맨',
 '분노의 질주: 더 얼티메이트',
 '백두산',
 '싱크홀',
 '극장판 귀멸의 칼날: 무한열차편',
 '베놈 2: 렛 데어 비 카니지',
 '소울',
 '테넷',
 '크루엘라',
 '#살아있다',
 '강철비2: 정상회담',
 '샹치와 텐 링즈의 전설',
 '담보',
 '인질',
 '닥터 두리틀',
 '삼진그룹 영어토익반',
 '듄',
 '도굴',
 '정직한 후보',
 '보이스',
 '클로젯',
 '007 노 타임 투 다이',
 '오케이 마담',
 '해치지않아',
 '천문: 하늘에 묻는다',
 '미나리',
 '미드웨이',
 '발신제한',
 '시동',
 '보스 베이비 2',
 '결백',
 '1917',
 '작은 아씨들',
 '콰이어트 플레이스 2',
 '랑종',
 '유체이탈자',
 '컨저링3: 악마가 시켰다',
 '기적',
 '고질라 VS. 콩',
 '지푸라기라도 잡고 싶은 짐승들',
 '엔칸토: 마법의 세계',
 '미스터 주: 사라진 VIP',
 '연애 빠진 로맨스',
 '인비저블맨',
 '킹스맨: 퍼스트 에이전트',
 '원더 우먼 1984',
 '나쁜 녀석들: 포에버',
 '국제수사',
 '침입자',
 '장르만 로맨스',
 '스타워즈: 라이즈 오브 스카이워커',
 '스파이 지니어스',
 '미션 파서블',
 '겨울왕국 2',
 '이웃사촌',
 '더 수어사이드 스쿼드',
 '온워드: 단 하루의 기적',
 '소리도 없이',
 '버즈 오브 프레이(할리 퀸의 황홀한 해방)',
 '비와 당신의 이야기',
 '서복',
 '킬러의 보디가드 2',
 '루카',
 '오! 문희',
 '자산어보',
 '내일의 기억',
 '그린랜드',
 '라야와 마지막 드래곤',
 '프리 가이',
 '더 스파이',
 '강릉',
 '위대한 쇼맨',
 '런',
 '정글 크루즈',
 '신비아파트 극장판 하늘도깨비 대 요르문간드',
 '뮬란',

In [14]:
body = []
score = []


In [71]:
from selenium.webdriver.common.keys import Keys

In [23]:
# pd파일 읽어서 영화명 리스트로 만들기
df = pd.read_csv('제목.csv', encoding='euc-kr')
movie_list = df['영화제목'].tolist()


In [ ]:
# pd파일 읽어서 영화명 리스트로 만들기
df = pd.read_csv('제목.csv', encoding='euc-kr')
movie_list = df['영화제목'].tolist()

# 데이터를 저장할 빈 리스트
bodies = []
URL = "https://movie.daum.net/main"
for movie in tqdm(movie_list):
    # 검색창에 영화제목 입력
    driver.get("https://movie.daum.net/search?q=" + movie)
    time.sleep(1)
    try:
        # 영화 소개 들어가기
        driver.find_element(By.XPATH, '//*[@id="mainContent"]/div/div[2]/div[2]/ul/li/div/div/strong/a').click()
        time.sleep(1)

        # 더보기 버튼이 있는지 확인
        more_button = driver.find_elements(By.XPATH, '//*[@id="mainContent"]/div/div[2]/div[2]/div[1]/div/div/a')

         # 더보기 버튼 클릭
        if more_button:
            more_button[0].click()
            time.sleep(1)

        # 영화 소개 텍스트 가져오기
        body_element = driver.find_element(By.XPATH, '//*[@id="mainContent"]/div/div[2]/div[2]/div[1]/div/div/div')
        bodies.append(body_element.text)
        
        # 영화 소개가 없으면 none 추가
    except Exception as e:
        bodies.append(None)
# 데이터를 DataFrame으로 변환
df = pd.DataFrame({
    '영화제목': movie_list,
    '소개': bodies
})

# CSV 파일로 저장
df.to_csv('info.csv', index=False)

In [7]:
# pd파일 읽어서 영화명 리스트로 만들기
df = pd.read_csv('17-23movis.csv')
movie_list = df['영화제목'].tolist()
movie_list  


# 데이터를 저장할 빈 리스트
bodies = []

URL = "https://movie.daum.net/main"

options = webdriver.ChromeOptions()
# options.add_argument("--headless") # 브라우저 안뜨게 하는 옵션
options.add_argument('--disable-dev-shm-usage') # 브라우저 메모리 관련 옵션 
options.add_argument("--no-sandbox") # 제한된 공간에서 실행되는 것을 허용하는 옵션

driver = webdriver.Chrome(options=options)
driver.set_window_size(1200, 2000)
driver.get(URL)
time.sleep(0.5)

for movie in tqdm(movie_list):
    # 검색창에 영화제목 입력
    driver.get("https://movie.daum.net/search?q=" + movie)
    time.sleep(1)

    try:
        # 영화 소개 들어가기
        driver.find_element(By.XPATH, '//*[@id="mainContent"]/div/div[2]/div[2]/ul/li/div/div/strong/a').click()
        time.sleep(1)

        # 더보기 버튼이 있는지 확인
        more_button = driver.find_elements(By.XPATH, '//*[@id="mainContent"]/div/div[2]/div[2]/div[1]/div/div/a')

         # 더보기 버튼 클릭
        if more_button:
            more_button[0].click()
            time.sleep(1)

        # 영화 소개 텍스트 가져오기
        body_element = driver.find_element(By.XPATH, '//*[@id="mainContent"]/div/div[2]/div[2]/div[1]/div/div/div')
        bodies.append(body_element.text)
        
        # 영화 소개가 없으면 none 추가
    except Exception as e:
        bodies.append(None)
  


# 데이터를 DataFrame으로 변환
df = pd.DataFrame({
    '영화제목': movie_list,
    '소개': bodies
})

# CSV 파일로 저장
df.to_csv('info.csv', index=False)

# Chrome WebDriver 종료
driver.quit()

print("데이터 저장이 완료되었습니다.")

100%|██████████| 3565/3565 [4:22:51<00:00,  4.42s/it]  


데이터 저장이 완료되었습니다.


In [33]:
# 데이터를 저장할 빈 리스트

URL = "https://www.naver.com"


options = webdriver.ChromeOptions()
# options.add_argument("--headless") # 브라우저 안뜨게 하는 옵션
options.add_argument('--disable-dev-shm-usage') # 브라우저 메모리 관련 옵션 
options.add_argument("--no-sandbox") # 제한된 공간에서 실행되는 것을 허용하는 옵션

driver = webdriver.Chrome(options=options)
driver.set_window_size(1200, 2000)
driver.get(URL)
time.sleep(0.5)


superv = []
lead = []

for movie in tqdm(movie_list[300:400]):
  
    # 검색창에 영화제목 입력
    search_text = "영화 "+ movie + " 출연진"
    driver.get("https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=" + search_text)
    time.sleep(1)

    # 영화 감독 가져오기
    super_element = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div/div[2]/div[1]/div/div/div/ul/li/a/div/div[2]/strong/span')
    superv.append(super_element.text)
    

    # 영화 주연 가져오기
    lead_element = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div/div[2]/div[2]/div/div/div')
    lead.append(lead_element.text)
  

# 현재 묶음의 데이터를 DataFrame으로 변환
df = pd.DataFrame({
    '영화제목' : movie_list[300:400],
    '감독': superv,
    '주연': lead
    
})

# CSV 파일로 저장
df.to_csv(f'movie_suv3.csv', index=False)

# Chrome WebDriver 종료
driver.quit()

print("데이터 저장이 완료되었습니다.")

100%|██████████| 100/100 [02:44<00:00,  1.65s/it]


데이터 저장이 완료되었습니다.


In [539]:
# 데이터를 저장할 빈 리스트
bodies = []
genres = []
scores = []

# Chrome WebDriver 시작
driver = webdriver.Chrome(options=options)
driver.set_window_size(1200, 2000)

# 30개씩 묶어서 처리
batch_size = 30

# 첫 번째 배치 (1~30)
start_index = 301
end_index = 350
current_movie_list = movie_list[start_index - 1:end_index]

bodies = []
genres = []
scores = []

for movie in tqdm(current_movie_list):
    try:
        # 검색창에 영화제목 입력
        search_text = "영화 " + movie + " 정보"
        driver.get("https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=" + search_text)
        time.sleep(0.5)

        # 영화 소개 가져오기
        body_element = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div[2]/div/div[2]/p')
        bodies.append(body_element.text)

        # 영화 장르 가져오기
        genre_element = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div[1]/div/div[1]/dl/div[3]/dd')
        genres.append(genre_element.text)

        # 평점 들어가기
        search_text = "영화 " + movie + " 평점"
        driver.get("https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=" + search_text)
        time.sleep(0.5)

        # 평점 가져오기
        score_element = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div[1]/div/div[2]/div/div/ul/li[1]/div/div[1]/span[1]')
        scores.append(score_element.text)

    except NoSuchElementException as e:
        print(f"Error occurred while processing {movie}: {e}")
        bodies.append(None)
        genres.append(None)
        scores.append(None)

# 현재 묶음의 데이터를 DataFrame으로 변환
df = pd.DataFrame({
    '영화제목': current_movie_list,
    '소개': bodies,
    '장르': genres,
    '평점': scores
})

# CSV 파일로 저장
df.to_csv(f'movie_data_batch_{start_index}_{end_index}.csv', index=False)

# Chrome WebDriver 종료
driver.quit()

print("데이터 저장이 완료되었습니다.")

  4%|▍         | 2/50 [00:19<07:49,  9.77s/it]


NameError: name 'NoSuchElementException' is not defined

In [ ]:

options = webdriver.ChromeOptions()
# options.add_argument("--headless") # 브라우저 안뜨게 하는 옵션
options.add_argument('--disable-dev-shm-usage') # 브라우저 메모리 관련 옵션 
options.add_argument("--no-sandbox") # 제한된 공간에서 실행되는 것을 허용하는 옵션

driver = webdriver.Chrome(options=options)
driver.set_window_size(1200, 2000)
driver.get(URL)
time.sleep(0.5)

# 리스트 리셋
bodies = []
scores = []
genres = []

# 첫 번째 영화만 가져와서 검색하기
first_movie = movie_list[0]

# 영화제목 리스트 가져와서 검색하기
for movie in tqdm(movie_list):
    
    # 검색창에 영화제목 입력
    search_text = "영화 " + movie + " 정보"  
    driver.get("https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=" + search_text)
    time.sleep(0.5)
                
    # 영화 소개 가져오기
    body_element = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div[2]/div/div[2]/p')
    bodies.append(body_element.text)
                
    # 영화 장르 가져오기
    genre_element = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div[1]/div/div[1]/dl/div[3]/dd')
    genres.append(genre_element.text)

    # 평점 들어가기
    search_text = "영화 " + movie + " 평점" 
    driver.get("https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=" + search_text)
    time.sleep(0.5)
                
    # 평점 가져오기
    scores = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div[1]/div/div[2]/div/div/ul/li[1]/div/div[1]/span[1]')
    score.append(scores.text)
    time.sleep(0.5)
   

driver.quit()

# 데이터프레임 생성
df = pd.DataFrame({
    '영화제목': movie_list,
    '소개': bodies,
    '장르': genres,
    '평점': scores
})

print(df)

 26%|██▌       | 112/437 [08:34<24:53,  4.59s/it]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main_pack"]/div[2]/div[2]/div[1]/div/div[2]/div/div/ul/li[1]/div/div[1]/span[1]"}
  (Session info: chrome=120.0.6099.71); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF612824D02+56194]
	(No symbol) [0x00007FF6127904B2]
	(No symbol) [0x00007FF6126376AA]
	(No symbol) [0x00007FF6126816D0]
	(No symbol) [0x00007FF6126817EC]
	(No symbol) [0x00007FF6126C4D77]
	(No symbol) [0x00007FF6126A5EBF]
	(No symbol) [0x00007FF6126C2786]
	(No symbol) [0x00007FF6126A5C23]
	(No symbol) [0x00007FF612674A45]
	(No symbol) [0x00007FF612675AD4]
	GetHandleVerifier [0x00007FF612B9D5BB+3695675]
	GetHandleVerifier [0x00007FF612BF6197+4059159]
	GetHandleVerifier [0x00007FF612BEDF63+4025827]
	GetHandleVerifier [0x00007FF6128BF029+687785]
	(No symbol) [0x00007FF61279B

In [ ]:


# 데이터 수집을 위한 리스트 초기화
bodies = []
scores = []
genres = []

for movie in tqdm(movie_list):
    try:
        # 영화 정보 검색
        search_text =  + movie + " 정보"
        driver.get("https://www.google.com/search?q=" + search_text)
        time.sleep(0.5)

        # 영화 소개 가져오기
        body_element = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div[2]/div/div[2]/p')
        bodies.append(body_element.text)
        
        # 영화 장르 가져오기
        genre_element = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div[1]/div/div[1]/dl/div[3]/dd')
        genres.append(genre_element.text)

        # 영화 평점 검색
        search_text = "영화 " + movie + " 평점"
        driver.get("https://www.google.com/search?q=" + search_text)
        time.sleep(0.5)

        # 평점 가져오기
        score_element = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div[1]/div/div[2]/div/div/ul/li[1]/div/div[1]/span[1]')
        scores.append(score_element.text)
    except Exception as e:
        print(f"Error occurred while processing {movie}: {e}")
        bodies.append(None)
        genres.append(None)
        scores.append(None)

driver.quit()

# 데이터프레임 생성
df = pd.DataFrame({
    '영화제목': movie_list,
    '소개': bodies,
    '장르': genres,
    '평점': scores
})

print(df)


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack


In [17]:
# 데이터 로드
df = pd.read_csv('3pj.csv')

# 필요한 컬럼 선택
df = df[['TFIDF', '매출액', '관객수']]  


C:\Users\User\AppData\Local\Temp\ipykernel_18816\2369138600.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('3pj.csv')


In [18]:

# 매출액 데이터 추가
sales_features = df[['매출액']]

In [19]:


# 타겟 변수
target = df['관객수']

In [89]:
X = df.copy()
X.drop(['관객수'], axis=1, inplace=True)
y = df['관객수']
X.shape, y.shape
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1530 entries, 0 to 1529
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TFIDF   1530 non-null   object
 1   매출액     1530 non-null   object
dtypes: object(2)
memory usage: 24.0+ KB


In [101]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42)


In [102]:
# TF-IDF 변환기 초기화 및 적용
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['TFIDF'])  # 'TFIDF' 컬럼에만 적용
X_test_tfidf = tfidf_vectorizer.transform(X_test['TFIDF'])


In [103]:
y_train = y_train.fillna('0')


In [104]:
y_train = y_train.dropna()


In [105]:
# 쉼표(,)를 제거하고 문자열을 숫자로 변환
y_train = y_train.str.replace(',', '').astype(float)

# NaN 값을 중앙값(median)으로 대체
median_value = y_train.median()
y_train = y_train.fillna(median_value)

# 최종적으로 정수형으로 변환
y_train = y_train.astype(int)


In [106]:
# y_train 열의 데이터 타입 확인
print(y_train.dtype)


int32


In [52]:
# 2. 매출액 및 관객수 데이터를 숫자형으로 변환 및 정제
import pandas as pd
y_train = y_train.astype(str).str.replace(',', '').astype(int)


In [98]:
import pandas as pd

# y_train의 데이터 형식 확인
print(y_train.dtype)

# 만약 y_train이 문자열로 되어있지 않다면, 문자열로 변환 후 처리
if y_train.dtype != 'str':
    y_train = y_train.astype(str).str.replace(',', '').astype(int)
else:
    y_train = y_train.str.replace(',', '').astype(int)


int32


In [67]:
# 쉼표(,)를 제거하고 문자열을 정수로 변환
y_train = y_train.str.replace(',', '', regex=True).astype(float).fillna(0).astype(int)


In [68]:
# 1. TF-IDF 데이터를 밀집 행렬로 변환
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
x_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# 2. 매출액 및 관객수 데이터를 숫자형으로 변환 및 정제
import pandas as pd
# 이미 정수로 변환된 경우, .str을 사용할 필요 없음
y_train = y_train.replace(',', '').astype(int)

# 3. 모델 선택 및 훈련 (예시: 랜덤 포레스트)
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=42)
model.fit(x_train_tfidf, y_train)


RandomForestRegressor(random_state=42)

In [107]:
import numpy as np

y_test = y_test.values  # Pandas Series를 NumPy 배열로 변환


In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
x_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
x_test_tfidf = tfidf_vectorizer.transform(X_test)  # 이 부분 추가

In [88]:
x_test_tfidf.shape

(3, 73898)

In [111]:
# 2. 매출액 및 관객수 데이터를 숫자형으로 변환 및 정제
import pandas as pd
y_train = y_train.replace(',', '').astype(int)


In [112]:
# 3. 모델 선택 및 훈련 (예시: 랜덤 포레스트)
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=42)
model.fit(x_train_tfidf, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [2, 1224]

In [109]:
# 4. 테스트 데이터에 대한 예측값 생성
y_pred = model.predict(x_test_tfidf)  # X_test 대신 x_test_tfidf를 사용

# y_pred의 길이를 확인
print(len(y_pred))  # 결과는 306이어야 함

ValueError: X has 2 features, but RandomForestRegressor is expecting 73898 features as input.

In [87]:
# 5. 모델 평가 (예시: 평균 제곱 오차 및 R-squared)
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 6. 결과 출력
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)


ValueError: Found input variables with inconsistent numbers of samples: [306, 3]